In [8]:
%load_ext autoreload
%autoreload 2

import datetime
import os

import pandas as pd
import s3fs
from pyarrow import parquet

# import helpers.hs3 as hs3
import im_v2.ig.data.client.historical_bars as imvidchiba

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#!aws s3 ls --profile default s3://iglp-core-data/ds/ext/bars/taq/v1.0-prod/60/

In [ ]:
root_path = "s3://iglp-core-data/ds/ext/bars/taq/v1.0-prod/60/"

# filesystem = s3fs.S3FileSystem(profile="default") if path.startswith('s3://') else None
# filesystem.ls(path)
# dataset = parquet.ParquetDataset(args.path, filesystem=filesystem)

s3 = s3fs.S3FileSystem(profile="default")

# s3.ls(path)

In [ ]:
# path = "s3://iglp-core-data/ds/ext/bars/taq/v1.0-prod/60/"

# #s3 = hs3.get_s3fs(aws_profile="saml-spm-sasm")
# s3 = hs3.get_s3fs(aws_profile="default")
# print(s3)
# #s3.ls("s3://iglp-core-data/")
# #dates = hs3.listdir(PATH, mode="non-recursive", aws_profile="saml-spm-sasm")

# dates = s3.ls(path)

In [ ]:
filesystem = s3fs.S3FileSystem()
igid = 11000
filters = []
for igid in range(igid, igid + 2000):
    filters.append([("igid", "=", igid)])
print(len(filters))
date = "20200106"
columns = "start_time end_time ticker close volume".split()

path = os.path.join(root_path, date, "data.parquet")
dataset = parquet.ParquetDataset(
    path, filesystem=filesystem, filters=filters, use_ligacy_dataset=False
)
# table = dataset.read()#columns=columns)
table = dataset.read(columns=columns)
df = table.to_pandas()
print(df.shape)
print(len(df["igid"].unique()))

In [ ]:
filesystem = s3fs.S3FileSystem()
igid = 10971
filters = [("igid", "=", igid)]
print(len(filters))
date = "20200106"
columns = "start_time end_time ticker close volume igid".split()

path = os.path.join(root_path, date, "data.parquet")
dataset = parquet.ParquetDataset(
    path, filesystem=filesystem, filters=filters, use_ligacy_dataset=False
)
# table = dataset.read()#columns=columns)
table = dataset.read(columns=columns)
df = table.to_pandas()
print(df.shape)
print(len(df["igid"].unique()))

In [ ]:
filesystem = s3fs.S3FileSystem()
date = "20210106"
columns = "start_time end_time ticker close volume igid".split()
filters = None
path = os.path.join(root_path, date, "data.parquet")
dataset = parquet.ParquetDataset(
    path, filesystem=filesystem, filters=filters, use_ligacy_dataset=False
)
# table = dataset.read()#columns=columns)
table = dataset.read(columns=columns)
df = table.to_pandas()
print(df.shape)
print(len(df["igid"].unique()))

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
imvidchiba.process_data(df)

In [ ]:
ticker_map = df[["igid", "ticker"]].groupby("igid").first()["ticker"]

ticker_map

In [ ]:
igid_by_trading_vol.head(100)["ticker"].to_csv("top_100_by_trading_volume.csv")

In [ ]:
df_tmp.loc[13684]

In [20]:
# igids = None
igids = [13684]
dates = [datetime.date(2021, 1, 6)]
columns = ["close", "volume", "ticker"]

df = imvidchiba.get_cached_bar_data_for_date_interval(
    igids, dates, columns=columns
)

Retrieving TAQ data:   0%|          | 0/1 [00:00<?, ?it/s]

In [21]:
print(imvidchiba.compute_bar_data_stats(df, igid))

min index=2021-01-06 09:01:00-05:00
max index=2021-01-06 16:59:00-05:00
num unique index=479
df.shape=(479, 4)
igids=None
found igids=1


In [22]:
df.head(3)

,close,volume,ticker,igid
end_time,,,,
2021-01-06 09:01:00-05:00,NaN,0,TSLA,13684
2021-01-06 09:02:00-05:00,NaN,0,TSLA,13684
2021-01-06 09:03:00-05:00,NaN,0,TSLA,13684


In [23]:
volume = df.groupby("igid")[["volume"]].sum()
price = df.groupby("igid")[["close"]].mean()
ticker_map = df.groupby("igid")[["ticker"]].first()

df_tmp = pd.concat([volume, price, ticker_map], axis=1)
print(df_tmp.head())

df_tmp["notional"] = df_tmp["volume"] * df_tmp["close"]

igid_by_trading_vol = df_tmp.sort_values(by="notional", ascending=False)

igid_by_trading_vol.head(100)

         volume       close ticker
igid                              
13684  41558750  764.893882   TSLA


,volume,close,ticker,notional
igid,,,,
13684,41558750,764.893882,TSLA,3.178803e+10
